In [1]:
from multiprocessing import Pool
import numpy as np
import random
from itertools import combinations as cb
import math
from copy import deepcopy as dc
from tqdm import tqdm

In [2]:
"""Evaluate Function """
class Evaluate:
    def __init__(self):
        None
    def evaluate(self,gen):
        None
    def check_dimentions(self,dim):
        None

"""Common Function"""
def random_search(n,dim):
    """
    create genes list
    input:{ n: Number of population, default=20
            dim: Number of dimension
    }
    output:{genes_list → [[0,0,0,1,1,0,1,...]...n]
    }
    """
    gens=[[0 for g in range(dim)] for _ in range(n)]
    for i,gen in enumerate(gens) :
        r=random.randint(1,dim)
        for _r in range(r):
            gen[_r]=1
        random.shuffle(gen)
    return gens

In [3]:
"""BDFA"""
def BDFA(Eval_Func,n=20,m_i=200,dim=None,minf=0,prog=False,mp=None):
    """
    input:{ Eval_Func: Evaluate_Function, type is class
            n: Number of population, default=20
            m_i: Number of max iteration, default=300
            minf: minimazation flag, default=0, 0=maximization, 1=minimazation
            dim: Number of feature, default=None
            prog: Do you want to use a progress bar?, default=False
            }
    output:{Best value: type float 0.967
            Best position: type list(int) [1,0,0,1,.....]
            Nunber of 1s in best position: type int [0,1,1,0,1] → 3
            }
    """
    estimate=Eval_Func().evaluate
    if dim==None:
        dim=Eval_Func().check_dimentions(dim)
    maxiter=m_i#500
    #flag=dr#True
    best_v=float("-inf") if minf == 0 else float("inf")
    best_p=[0]*dim
    gens_dict={tuple([0]*dim):float("-inf") if minf == 0 else float("inf")}

    enemy_fit=float("-inf") if minf == 0 else float("inf")
    enemy_pos=[0 for _ in range(dim)]
    food_fit=float("-inf") if minf == 0 else float("inf")
    food_pos=[0 for _ in range(dim)]

    fit=[0 for _ in range(n)]
    genes=random_search(n,dim)
    genesX=random_search(n,dim)

    if prog:
        miter=tqdm(range(m_i))
    else:
        miter=range(m_i)
    for it in miter:
        w=0.9 - it * ((0.9-0.4) / maxiter)
        mc=0.1- it * ((0.1-0) / (maxiter/2))
        if mc < 0:
            mc=0
        s=2 * random.random() * mc
        a=2 * random.random() * mc
        c=2 * random.random() * mc
        f=2 * random.random()
        e=mc
        if it > (3*maxiter/3):
            e=0

        if mp!=None:
            fit=[gens_dict[tuple(g)]  if tuple(g) in gens_dict  else(float("-inf") if minf == 0 else float("inf")) for g in genes]
            alter_gens=[i for i,g in enumerate(genes) if tuple(g) not in gens_dict]
            with Pool(mp) as p:
                alter_fit = p.map(estimate,[genes[i] for i in alter_gens])
            z=0
            for i in range(len(fit)):
                if i in alter_gens:
                    gens_dict[tuple(genes[i])]=alter_fit[z]
                    fit[i]=alter_fit[z]
                    z+=1
            for i in range(n):
                if fit[i] > food_fit if minf==0 else fit[i] < food_fit:
                    food_fit=dc(fit[i])
                    food_pos=dc(genes[i])

                if fit[i] > enemy_fit if minf==0 else fit[i] < enemy_fit:
                    enemy_fit=dc(fit[i])
                    enemy_pos=dc(genes[i])
        else:
            for i in range(n):
                if tuple(genes[i]) in gens_dict:
                    fit[i]=gens_dict[tuple(genes[i])]
                else:
                    fit[i]=estimate(genes[i])
                    gens_dict[tuple(genes[i])]=dc(fit[i])
                if fit[i] > food_fit if minf==0 else fit[i] < food_fit:
                    food_fit=dc(fit[i])
                    food_pos=dc(genes[i])

                if fit[i] > enemy_fit if minf==0 else fit[i] < enemy_fit:
                    enemy_fit=dc(fit[i])
                    enemy_pos=dc(genes[i])

        for i in range(n):
            ind=-1
            nn=-1
            ndx=[[0 for _d in range(dim)] for _ in range(n)]
            nx=[[0 for _d in range(dim)] for _ in range(n)]

            for j in range(n):
                if i==j:
                    pass
                else:
                    ind+=1
                    nn+=1
                    ndx[ind]=dc(genesX[j])
                    nx[ind]=dc(genes[j])

            S=[0 for _ in range(dim)]
            for k in range(nn):
                S=[_s+(_x-_y) for _s,(_x,_y) in zip(S,zip(ndx[k],genes[i]))] #s+(nx[k]-x[i])
            S=S

            A=[sum([_[_d] for _ in ndx])/nn for _d in range(dim)]#[sum(_)/nn if _ != 0 else 0 for _ in ndx]
            #[_-g for _,g in zip([sum([_[_d] for _ in nx])/nn for _d in range(dim)],genes[i])]
            C=[_-g for _,g in zip([sum([_[_d] for _ in nx])/nn for _d in range(dim)],genes[i])]#[sum(_)/nn-g if _ != 0 else 0 for _,g in zip(nx,genes[i])]

            F=[fp-g for fp,g in zip(food_pos,genes[i])]
            E=[ep+g for  ep,g in zip(enemy_pos,genes[i])]

            for j in range(dim):
                genesX[i][j]=s*S[j]+a*A[j]+c*C[j]+ f *F[j]+e*E[j]+w*genesX[i][j]

                if genesX[i][j] > 6:
                    genesX[i][j]=6
                if genesX[i][j] < -6:
                    genesX[i][j]=-6
                T = abs(genesX[i][j] / math.sqrt((1+genesX[i][j]**2)))
                if random.random()<T:
                    genes[i][j]=1 if genes[i][j] == 0 else 0
    best_p=food_pos
    best_v=food_fit
    print(len(gens_dict))
    return best_v,best_p,best_p.count(1)

In [4]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
 
from sklearn.datasets import load_boston
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

import numpy as np

from sklearn import svm
 
SEED = 2018
np.random.seed(SEED)
df_train = pd.read_csv("VoxForge512.csv")
X = np.array(df_train.drop(['label'],axis=1))
y = np.array(df_train['label'])
from sklearn.model_selection import train_test_split
df=df_train
(a,b)=np.shape(df)
print(a,b)
data = df.values[:,0:b-1]
label = df.values[:,b-1]
# trainX=data
# trainy=label
cross=4
test_size=(1/cross)
trainX, testX, trainy, testy = train_test_split(data, label,stratify=label ,test_size=test_size)

900 201


In [5]:
if __name__ == '__main__':
    from sklearn import svm
    from time import time

    np.random.seed(20)
    tr_d=trainX
    te_d=testX
    tr_l=trainy
    te_l=testy

In [6]:
class Evaluate:#setting class
        def __init__(self):#set train_data,label,test_data,label
            self.train_l=tr_l
            self.train_d=tr_d
            self.test_l=te_l
            self.test_d=te_d
        def evaluate(self,gen):
            """
            Setting of evaluation function.
            Here, the correct answer rate is used.
              anser_label/all_label
            """
            mask=np.array(gen) > 0
            al_data=np.array([al[mask] for al in self.train_d])
            al_test_data=np.array([al[mask] for al in self.test_d])
            #↑masking with [01]sequence list
            res=RandomForestClassifier(n_jobs = -1, verbose = 0, n_estimators=5, criterion='entropy', random_state = 0).fit(al_data,self.train_l).predict(al_test_data)
            return np.count_nonzero(self.test_l==res)/len(self.test_l)
            #↑evaluate with fittness function
        def check_dimentions(self,dim):#check number of all feature
            if dim==None:
                return len(self.train_d[0])
            else:
                return dim

In [7]:
%%time
s,g,l=BDFA(Eval_Func=Evaluate, n=1, m_i=1)

2
CPU times: user 59.7 ms, sys: 7.35 ms, total: 67 ms
Wall time: 238 ms


In [8]:
l

129

In [9]:
List = []

for i in range(0,len(g)):
    if g[i] == 1:
        List.append(i)
        
df_train = pd.read_csv("VoxForge512.csv")
y = (df_train['label'])
df_train1 = df_train[df_train.columns[List]]
l1 = []
for i in range(0,len(List)):
    l1.append(i)
    
df_train1.columns = l1

df_train1['label'] = y
df_train1

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,0,1,2,3,4,5,6,7,8,9,...,120,121,122,123,124,125,126,127,128,label
0,0.003289,0.003680,0.002809,0.002662,0.002827,0.002014,0.004380,0.001103,0.002665,0.002319,...,6.411717,15.391540,14.609205,6.368087,14.887147,17.031933,0.016049,0.031459,0.009924,English
1,0.002735,0.002047,0.002489,0.002068,0.002175,0.002427,0.001584,0.000306,0.002996,0.001032,...,4.699897,7.948945,9.241147,8.133885,15.687752,17.947880,0.014899,0.029206,0.014316,English
2,0.001556,0.001723,0.001227,0.001315,0.001594,0.001637,0.001206,0.000508,0.001195,0.001047,...,6.692659,8.274411,10.038836,13.792500,17.805973,20.371273,0.014856,0.029121,0.015950,English
3,0.007180,0.010772,0.011593,0.007111,0.014183,0.009923,0.008713,0.002654,0.008530,0.003775,...,0.146815,0.284989,0.475082,4.076348,1.745809,1.997327,0.011871,0.023270,0.010955,English
4,0.003632,0.004742,0.003760,0.002512,0.004510,0.003478,0.004527,0.002201,0.003138,0.001852,...,2.625072,5.087078,5.555333,4.520422,5.230475,5.984027,0.011261,0.022074,0.008553,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,0.000754,0.000790,0.000955,0.000608,0.001347,0.000515,0.000559,0.000482,0.000726,0.000330,...,0.009261,0.027846,0.049385,0.001004,0.001476,0.001688,0.032924,0.064538,0.026434,Spanish
896,0.000093,0.000087,0.000061,0.000111,0.000110,0.000210,0.000161,0.000117,0.000092,0.000101,...,0.006949,0.012645,0.013272,0.002488,0.001386,0.001586,0.005506,0.010794,0.003441,Spanish
897,0.000122,0.000108,0.000083,0.000059,0.000091,0.000052,0.000040,0.000025,0.000042,0.000049,...,0.001491,0.002771,0.006380,0.000358,0.000601,0.000688,0.006557,0.012853,0.005301,Spanish
898,0.000005,0.000010,0.000008,0.000005,0.000006,0.000008,0.000005,0.000009,0.000008,0.000007,...,0.002749,0.003097,0.002084,0.000236,0.000454,0.000519,0.004320,0.008469,0.003344,Spanish


In [10]:
df_train1.to_csv('VF_BDFA.csv')